# **기업과제4_10팀_LSATF-IDFSVD**.ipynb
본 노트북은 요약 모델이 실제로 **본문의 주제**를 잘 감지하고 있는지 정성적으로 검증하기 위해 작성되었습니다.   
모델의 전반적인 학습과정이 궁금하시다면 `기업과제4_10팀.ipynb`을,
모델을 정량적으로 평가한 메트릭을 원하시면 `metric.py`을 확인해주시길 바랍니다.
```
⚓ 원티드 프리온보딩 10팀 / 강상훈, 권예환, 남준우, 장수림, 조성빈
```

# Appendix: LSATF-IDFSVD

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

In [ ]:
df = pd.read_csv("./sports_news_data.csv")
df.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


In [ ]:
import numpy as np
def HTMLCleaner(doc):
  """ 본문에서 불필요한 HTML Tags를 제거하고 문장을 나눕니다. """
  doc = re.sub('\n', '<p>', doc)                # Unify Newline Character
  doc = re.sub('\.', '.<p>', doc)
  soup = BeautifulSoup(doc, "html.parser")      # Handling HTML tags
  blocklist = ['span', 'strong', 'blockquote']  # Remove specific tags
  for tag in blocklist:
    tagging_soup = soup.findAll(tag)
    for script in tagging_soup:
      script.decompose()
  return [sentence for sentence in soup.strings]

def DocCleaner(text):
  """ 개별 문서에 적용해 문서를 전처리하는 함수입니다. """
  doc = HTMLCleaner(text)
  if doc:
    if doc[0].startswith('['):                # Remove media source
      doc[0] = re.sub('\[.*?\] ', '', doc[0])
      doc[0] = re.sub(r'^.*?= ', '', doc[0])  # Remove reporter name

    if doc[-1].startswith(('사진', '그래픽')):   # Remove photo source
      doc = doc[:-1]

  doc = [x.strip() for x in doc if x]       # Remove empty string in sentence list
  doc = [x for x in doc if x]
  doc = [x for x in doc if x != ' '] 

  # TODO: 인용문("") 처리
  return doc

def Preprocessing(df, col='CONTENT'):
  """ 전체 데이터프레임에 적용해 문서를 전처리하는 함수입니다. """
  # Drop NaN records
  df = df.dropna().reset_index(drop=True)
  # Cleaning the documents
  df['TEXT'] = df[col].apply(lambda x : DocCleaner(x))
  # Removing Outlier (using IQR)
  df['No_of_SENT'] = df['TEXT'].apply(lambda x : len(x))
#   q25 = np.quantile(df['No_of_SENT'], 0.25)
#   q75 = np.quantile(df['No_of_SENT'], 0.75)
#   iqr = (q75 - q25)
#   upper = q75 + (iqr * 1.5)
#   lower = q25 - (iqr * 1.5)
#   df = df[(df['No_of_SENT'] > lower) & (df['No_of_SENT'] < upper)].reset_index(drop=True)

  return df[['TITLE', 'TEXT', 'PUBLISH_DT']]

new_df = Preprocessing(df, col='CONTENT')

In [ ]:
from tqdm import tqdm

In [ ]:
from pororo import Pororo
summ = Pororo(task="summarization", model="abstractive", lang="ko")
summed = []
for text in tqdm(new_df.TEXT):
    summed.append(summ(' '.join(text)))

100%|████████████████████████████████████████████████████████████████████████████| 9075/9075 [2:30:00<00:00,  1.01it/s]


In [ ]:
new_df['summary'] = summed

In [ ]:
new_df['summary'].to_csv('./pororo-abstractive-summary.csv')

In [ ]:
summed = pd.read_csv('./pororo-abstractive-summary.csv')
new_df['summary'] = summed.summary

In [ ]:
new_df.head()

,TITLE,TEXT,PUBLISH_DT,summary
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마...,2021-07-15,유럽 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ‘판정 논란’...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명","[유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까., ...",2021-07-15,영국에서 시작된 유로와 코파 아메리카가 11일(한국 시간) 끝난 후 공식 베스트11...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,"[새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다., 영국 매체 ‘스포...",2021-07-15,영국 매체 '스포츠 바이블'은 맨체스터 시티가 승점 88을 쌓아 경쟁 팀들을 손쉽게...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원","[홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다., 이재성은...",2021-07-15,"전북 여름 전북 현대를 떠난 이재성에게 독일 분데스리가 마인츠, 호펜하임을 비롯해 ..."
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”","[FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다., 팬들은 만족스럽다는 반응이다...",2021-07-15,FC 바르셀로나가 15일 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨이 ...


## Mecab으로 Tokenizing 후 Tf-idf + Truncated SVD 적용

In [ ]:
from eunjeon import Mecab
mecab = Mecab()

In [ ]:
tokenized_text = []
for text in new_df.TEXT:
    tokenized_text.append(' '.join(mecab.morphs(' '.join(text))))

In [ ]:
tokenized_text[0]

'유럽 축구 연맹 ( UEFA ) 유로 2020 심판 위원장 로 베르토 로세티 가 잉글랜드 와 덴마크 전 에 나온 ‘ 판정 논란 ’ 을 정심 이 라고 공언 했 다 . 지난 8 일 ( 한국 시간 ) 잉글랜드 와 덴마크 는 유로 2020 4 강 에서 결승 티켓 을 두 고 격돌 했 다 . 연장 접전 끝 에 잉글랜드 가 2 - 1 로 이겼 다 . 하지만 경기 후 논란 이 불거졌 다 . 1 - 1 로 팽팽 하 던 연장 전반 12 분 라힘 스털링 이 드리블 돌파 를 하 던 중 요아킴 멜레 와 마티아스 옌센 사이 에서 넘어졌 다 . 심판 은 곧장 페널티 스팟 을 찍 었 다 . 비디오 판 독실 과 의견 을 나눈 뒤 에 도 원심 을 유지 했 다 . 페널티킥을 얻 은 잉글랜드 는 해리 케인 이 실축 했 지만 , 흐른 볼 을 밀 어 넣 어 결승 티켓 을 따 냈 다 . 이 장면 을 두 고 갑론을박 이 펼쳐졌 다 . 스털링 은 경기 후 인터뷰 에서 “ 명백 한 페널티킥 ” 이 라고 주장 했 지만 , 전문가 들 의 의견 은 달랐 다 . 조제 모리뉴 AS 로마 감독 , 아르센 벵거 전 아스널 감독 은 “ 페널티킥이 아니 다 ” 고 입 을 모았 다 . 여전히 많 은 이야기 가 흘러나오 는 가운데 , 유로 2020 심판 위원장 로세티 가 오심 이 아니 라는 입장 을 내놨 다 . 로세티 위원장 은 14 일 영국 매체 ‘ 가디언 ’ 과 인터뷰 에서 “ 주심 은 5 번 수비수 ( 멜레 ) 를 주목 했 다 . 수비수 가 볼 을 터치 하 지 않 았 다고 봤 다 . 멜레 의 오른쪽 다리 와 스털링 의 오른쪽 다리 와 접촉 한 것 으로 확인 했 다 . 물론 접촉 강도 를 논할 수 있 지만 , 우리 는 항상 심판 이 의사 결정 과정 의 중심 이 되 길 바란다 ” 고 밝혔 다 . 로세티 위원장 이 심판 을 대표 해 의견 을 냈 지만 , 오심 이 라고 생각 하 는 이 들 이 받아들일지 는 미지수 다 . 스털링 은 평소 에 도 ‘ 다이빙 논란 ’ 에 시달려왔 고 , 많 은 이 들 의 머릿속 에 ‘ 다이버 ’ 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, max_df = 0.5, smooth_idf=True)
tfidf_matrix = vectorizer.fit_transform(tokenized_text)
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(tfidf_matrix)

TruncatedSVD(n_components=20, n_iter=100, random_state=122)

## 기사 제목(Title)에 적용 시 1278

In [ ]:
tokenized_title = []
for title in new_df.TITLE:
    tokenized_title.append(' '.join(mecab.morphs(title)))

In [ ]:
anchor_ = svd_model.transform(vectorizer.transform(tokenized_title))

In [ ]:
np.sum(anchor_)

1278.9111963083446

## Pororo 생성 요약

In [ ]:
tokenized_abs_summary = []
for text in new_df.summary:
    tokenized_abs_summary.append(' '.join(mecab.morphs(text)))

In [ ]:
tokenized_abs_summary[0]

'유럽 2020 심판 위원장 로 베르토 로세티 가 잉글랜드 와 덴마크 전 에 나온 ‘ 판정 논란 ’ 을 정심 이 라고 공언 했 지만 많 은 사람 들 이 받아들일지 는 미지수 다 .'

In [ ]:
abs_ = svd_model.transform(vectorizer.transform(tokenized_abs_summary))
np.sum(abs_)

1191.271032488458

## kobart 요약본

In [ ]:
df = pd.read_csv('data_with_summary.csv')
tokenized_kobart = []
for summary in df.SUMMARY:
    tokenized_kobart.append(' '.join(mecab.morphs(summary)))

In [ ]:
kobart_ = svd_model.transform(vectorizer.transform(tokenized_kobart))
np.sum(kobart_)

1156.505019970497

## Pororo 추출 요약

In [ ]:
df = pd.read_csv('pororo_ext.csv')
tokenized_por_ext = []
for summary in df.summary:
    try:
        tokenized_por_ext.append(' '.join(mecab.morphs(summary)))
    except NotImplementedError as e:
        print(e)
        print('summary : ', summary)
ext_ = svd_model.transform(vectorizer.transform(tokenized_por_ext))
np.sum(ext_)

Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)

summary :  nan
Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)

summary :  nan


1461.8876849319715

# kobertsum 추출 요약

In [ ]:
df = pd.read_csv('result.csv')
tokenized_bertsum = []
for summary in df.summary:
    try:
        tokenized_bertsum.append(' '.join(mecab.morphs(summary)))
    except NotImplementedError as e:
        print(e)
        print('summary : ', summary)
bertsum_ = svd_model.transform(vectorizer.transform(tokenized_bertsum))
np.sum(bertsum_)

Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)

summary :  nan
Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)

summary :  nan


1443.8575572582256